# Ingesting Data

### Introduction

Now that we have a sense of the flow of our data, let's work on querying that data.

### Connecting to Postgres

The first thing we'll need to do is connect to postgres.

In [ ]:
# FILL IN THE REDSHIFT ENPOINT HERE
# e.g. DWH_ENDPOINT="redshift-cluster-1.csmamz5zxmle.us-west-2.redshift.amazonaws.com" 
DWH_ENDPOINT="redshift-cluster-1.cdpgnoufdsdf.us-east-1.redshift.amazonaws.com" 
    
#FILL IN THE IAM ROLE ARN you got in step 2.2 of the previous exercise
#e.g DWH_ROLE_ARN="arn:aws:iam::988332130976:role/dwhRole"
DWH_ROLE_ARN="arn:aws:iam::095598444804:role/redshiftRole"

conn_string="postgresql://{}:{}@{}:{}/{}".format("awsuser", "Password1", DWH_ENDPOINT, DWH_PORT,"dev")

> To get data from S3, we need to specify the access key and secret access key to allow us to read the data.

Notice we signed up for this before.
> <img src="./s3_role.png" width="60%">

> So here we create data with the credentials from this role.

In [ ]:
import boto3

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                     )
sampleDbBucket =  s3.Bucket("udacity-labs")

for obj in sampleDbBucket.objects.filter(Prefix="tickets"):
    print(obj)


And we'll need to use the role ARN with S3 access to copy the data into redshift. 

In [1]:
%%time
qry = """
    copy restaurants from 's3://sample-sample-data/yelp-lunch-nyc.csv'
    credentials 'aws_iam_role=arn:aws:iam::095598444804:role/myRedshiftRole'
     region 'us-east-1';
""".format("arn:aws:iam::095598444804:role/myRedshiftRole")

%sql $qry

UsageError: Line magic function `%sql` not found.


```bash
InternalError: (psycopg2.InternalError) User arn:aws:redshift:us-east-1:095598444804:dbuser:redshift-cluster-1/awsuser is not authorized to assume IAM Role arn:aws:iam::095598444804:role/myRedshiftRole. 
DETAIL:  
  -----------------------------------------------
  error:  User arn:aws:redshift:us-east-1:095598444804:dbuser:redshift-cluster-1/awsuser is not authorized to assume IAM Role arn:aws:iam::095598444804:role/myRedshiftRole. 
```

But we need this with a role that redshift has access to.  

### Resources

[Boto S3](https://realpython.com/python-boto3-aws-s3/)

[S3 tutorial](http://boto.cloudhackers.com/en/latest/s3_tut.html)

[Saving from RDS to S3 (really aurora)](https://medium.com/@max.borysov/how-to-migrate-data-between-aws-rds-and-s3-1a011c8aca52)

[Good tutorial RDS to S3](https://www.sqlshack.com/integrating-aws-s3-buckets-with-aws-rds-sql-server/)